<a href="https://colab.research.google.com/github/thnavear/Qualcomm-DL-Hackathon/blob/master/genai_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.3 MB/s eta 0:00:00


In [2]:
from groq import Groq

client = Groq(api_key="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")
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Hello"}], # Added a user message
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


#os.environ["TOGETHER_API_KEY"]="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"

Hello. How can I assist you today?